### Universidad Del Valle de Guatemala
### Departamento de computación
### Modelación y Simulación
### Proyecto
### Diana Lucía Fernández Villatoro - 21747
### Daniel Esteban Morales Urizar - 21785
### Brandon Rolando Sicay Cumes - 21757

Simulación de un triaje médico y atención médica dependiendo de las prioridades de los pacientes sin posibilidad de interrupción a la atención de un paciente. UN MÉDICO

In [4]:
import heapq
import random
import time

# Definimos las prioridades de triaje
PRIORIDADES = {
    "I": {"descripcion": "Prioridad absoluta, atención inmediata", "demora_maxima": 0},
    "II": {"descripcion": "Muy urgente, riesgo vital", "demora_maxima": 15},
    "III": {"descripcion": "Urgente pero estable, potencial riesgo vital", "demora_maxima": 60},
    "IV": {"descripcion": "Urgencia menor, sin riesgo vital", "demora_maxima": 120}
}

# Clase para representar a un paciente
class Paciente:
    def __init__(self, nombre, prioridad):
        self.nombre = nombre
        self.prioridad = prioridad
        self.tiempo_llegada = time.time()  # Para ordenar en caso de prioridades iguales

    def __lt__(self, other):
        return (self.prioridad, self.tiempo_llegada) < (other.prioridad, other.tiempo_llegada)

    def __str__(self):
        return f"{self.nombre} - Prioridad {self.prioridad}: {PRIORIDADES[self.prioridad]['descripcion']}"

# Clase para manejar eventos en la simulación
class Evento:
    def __init__(self, tiempo, tipo, paciente=None):
        self.tiempo = tiempo  # Momento en el que ocurre el evento
        self.tipo = tipo  # "llegada", "inicio_atencion", "fin_atencion"
        self.paciente = paciente

    def __lt__(self, other):
        return self.tiempo < other.tiempo

# Clase para el centro de salud que maneja los eventos y la cola de pacientes
class CentroSalud:
    def __init__(self):
        self.cola_pacientes = []
        self.eventos = []
        self.tiempo_actual = 0
        self.doctor_ocupado = False

    def agendar_evento(self, evento):
        heapq.heappush(self.eventos, evento)

    def llegada_paciente(self, paciente):
        print(f"Tiempo {self.tiempo_actual}: Llega {paciente}")
        heapq.heappush(self.cola_pacientes, paciente)

        # Si el doctor está libre, iniciamos atención inmediata
        if not self.doctor_ocupado:
            self.iniciar_atencion()

    def iniciar_atencion(self):
        if self.cola_pacientes:
            paciente = heapq.heappop(self.cola_pacientes)
            self.doctor_ocupado = True
            tiempo_atencion = random.randint(5, 20)  # Simula tiempo de atención
            fin_atencion = self.tiempo_actual + tiempo_atencion
            print(f"Tiempo {self.tiempo_actual}: Inicia atención para {paciente}, tiempo estimado {tiempo_atencion} minutos")
            self.agendar_evento(Evento(fin_atencion, "fin_atencion", paciente))

    def fin_atencion(self, paciente):
        print(f"Tiempo {self.tiempo_actual}: Finaliza atención para {paciente}")
        self.doctor_ocupado = False
        # Si hay más pacientes en la cola, atender al siguiente
        self.iniciar_atencion()

    def procesar_eventos(self, tiempo_maximo):
        while self.eventos and self.tiempo_actual <= tiempo_maximo:
            evento = heapq.heappop(self.eventos)
            self.tiempo_actual = evento.tiempo
            if evento.tipo == "llegada":
                self.llegada_paciente(evento.paciente)
            elif evento.tipo == "inicio_atencion":
                self.iniciar_atencion()
            elif evento.tipo == "fin_atencion":
                self.fin_atencion(evento.paciente)

    def simular(self, tiempo_maximo):
        # Genera eventos de llegada de pacientes de manera aleatoria en el tiempo
        nombres = ["Paciente A", "Paciente B", "Paciente C", "Paciente D", "Paciente E"]
        prioridades = ["I", "II", "III", "IV"]
        
        for i in range(10):  # Genera algunos pacientes de prueba
            tiempo_llegada = random.randint(1, tiempo_maximo)
            nombre = random.choice(nombres)
            prioridad = random.choice(prioridades)
            paciente = Paciente(nombre, prioridad)
            self.agendar_evento(Evento(tiempo_llegada, "llegada", paciente))
        
        self.procesar_eventos(tiempo_maximo)

# Simulación
tiempo_simulacion = 500  # Simular 100 minutos
centro_salud = CentroSalud()
centro_salud.simular(tiempo_simulacion)


Tiempo 22: Llega Paciente A - Prioridad III: Urgente pero estable, potencial riesgo vital
Tiempo 22: Inicia atención para Paciente A - Prioridad III: Urgente pero estable, potencial riesgo vital, tiempo estimado 10 minutos
Tiempo 32: Finaliza atención para Paciente A - Prioridad III: Urgente pero estable, potencial riesgo vital
Tiempo 125: Llega Paciente E - Prioridad IV: Urgencia menor, sin riesgo vital
Tiempo 125: Inicia atención para Paciente E - Prioridad IV: Urgencia menor, sin riesgo vital, tiempo estimado 11 minutos
Tiempo 136: Finaliza atención para Paciente E - Prioridad IV: Urgencia menor, sin riesgo vital
Tiempo 162: Llega Paciente D - Prioridad III: Urgente pero estable, potencial riesgo vital
Tiempo 162: Inicia atención para Paciente D - Prioridad III: Urgente pero estable, potencial riesgo vital, tiempo estimado 15 minutos
Tiempo 177: Finaliza atención para Paciente D - Prioridad III: Urgente pero estable, potencial riesgo vital
Tiempo 181: Llega Paciente D - Prioridad II

Simulación de un triaje médico y atención médica dependiendo de las prioridades de los pacientes con posibilidad de interrupción a la atención de un paciente. UN MÉDICO

In [10]:
import heapq
import random
import time

# Definimos las prioridades de triaje
PRIORIDADES = {
    "I": {"descripcion": "Prioridad absoluta, atención inmediata", "demora_maxima": 0},
    "II": {"descripcion": "Muy urgente, riesgo vital", "demora_maxima": 15},
    "III": {"descripcion": "Urgente pero estable, potencial riesgo vital", "demora_maxima": 60},
    "IV": {"descripcion": "Urgencia menor, sin riesgo vital", "demora_maxima": 120}
}

# Clase para representar a un paciente
class Paciente:
    def __init__(self, nombre, prioridad):
        self.nombre = nombre
        self.prioridad = prioridad
        self.tiempo_llegada = time.time()  # Para ordenar en caso de prioridades iguales

    def __lt__(self, other):
        return (self.prioridad, self.tiempo_llegada) < (other.prioridad, other.tiempo_llegada)

    def __str__(self):
        return f"{self.nombre} - Prioridad {self.prioridad}: {PRIORIDADES[self.prioridad]['descripcion']}"

# Clase para manejar eventos en la simulación
class Evento:
    def __init__(self, tiempo, tipo, paciente=None):
        self.tiempo = tiempo  # Momento en el que ocurre el evento
        self.tipo = tipo  # "llegada", "inicio_atencion", "fin_atencion"
        self.paciente = paciente

    def __lt__(self, other):
        return self.tiempo < other.tiempo

# Clase para el centro de salud que maneja los eventos y la cola de pacientes
class CentroSalud:
    def __init__(self):
        self.cola_pacientes = []
        self.eventos = []
        self.tiempo_actual = 0
        self.doctor_ocupado = False
        self.paciente_actual = None  # Paciente que está siendo atendido actualmente

    def agendar_evento(self, evento):
        heapq.heappush(self.eventos, evento)

    def llegada_paciente(self, paciente):
        print(f"Tiempo {self.tiempo_actual}: Llega {paciente}")
        
        # Si el doctor está libre, iniciamos atención inmediata
        if not self.doctor_ocupado:
            self.iniciar_atencion(paciente)
        else:
            # Interrumpir si el nuevo paciente tiene mayor prioridad que el paciente actual
            if self.paciente_actual and paciente.prioridad < self.paciente_actual.prioridad:
                print(f"Tiempo {self.tiempo_actual}: {paciente} interrumpe la atención de {self.paciente_actual}")
                # Regresar al paciente actual a la cola
                heapq.heappush(self.cola_pacientes, self.paciente_actual)
                # Atender al nuevo paciente
                self.iniciar_atencion(paciente)
            else:
                # Agregar el nuevo paciente a la cola
                heapq.heappush(self.cola_pacientes, paciente)

    def iniciar_atencion(self, paciente):
        self.paciente_actual = paciente
        self.doctor_ocupado = True
        tiempo_atencion = random.randint(5, 20)  # Simula tiempo de atención
        fin_atencion = self.tiempo_actual + tiempo_atencion
        print(f"Tiempo {self.tiempo_actual}: Inicia atención para {paciente}, tiempo estimado {tiempo_atencion} minutos")
        self.agendar_evento(Evento(fin_atencion, "fin_atencion", paciente))

    def fin_atencion(self, paciente):
        if paciente == self.paciente_actual:
            print(f"Tiempo {self.tiempo_actual}: Finaliza atención para {paciente}")
            self.paciente_actual = None
            self.doctor_ocupado = False
            # Si hay más pacientes en la cola, atender al siguiente
            if self.cola_pacientes:
                siguiente_paciente = heapq.heappop(self.cola_pacientes)
                self.iniciar_atencion(siguiente_paciente)

    def procesar_eventos(self, tiempo_maximo):
        while self.eventos and self.tiempo_actual <= tiempo_maximo:
            evento = heapq.heappop(self.eventos)
            self.tiempo_actual = evento.tiempo
            if evento.tipo == "llegada":
                self.llegada_paciente(evento.paciente)
            elif evento.tipo == "fin_atencion":
                self.fin_atencion(evento.paciente)

    def simular(self, tiempo_maximo):
        # Genera eventos de llegada de pacientes de manera aleatoria en el tiempo
        nombres = ["Paciente A", "Paciente B", "Paciente C", "Paciente D", "Paciente E", "Paciente 1", "Paciente 2"]
        prioridades = ["I", "II", "III", "IV"]
        
        for i in range(5):  # Genera algunos pacientes de prueba
            tiempo_llegada = random.randint(1, tiempo_maximo)
            nombre = random.choice(nombres)
            prioridad = random.choice(prioridades)
            paciente = Paciente(nombre, prioridad)
            self.agendar_evento(Evento(tiempo_llegada, "llegada", paciente))
        
        self.procesar_eventos(tiempo_maximo)

# Simulación
tiempo_simulacion = 100  # Simular 100 minutos
centro_salud = CentroSalud()
centro_salud.simular(tiempo_simulacion)


Tiempo 8: Llega Paciente B - Prioridad III: Urgente pero estable, potencial riesgo vital
Tiempo 8: Inicia atención para Paciente B - Prioridad III: Urgente pero estable, potencial riesgo vital, tiempo estimado 19 minutos
Tiempo 10: Llega Paciente 1 - Prioridad IV: Urgencia menor, sin riesgo vital
Tiempo 27: Finaliza atención para Paciente B - Prioridad III: Urgente pero estable, potencial riesgo vital
Tiempo 27: Inicia atención para Paciente 1 - Prioridad IV: Urgencia menor, sin riesgo vital, tiempo estimado 17 minutos
Tiempo 32: Llega Paciente 2 - Prioridad I: Prioridad absoluta, atención inmediata
Tiempo 32: Paciente 2 - Prioridad I: Prioridad absoluta, atención inmediata interrumpe la atención de Paciente 1 - Prioridad IV: Urgencia menor, sin riesgo vital
Tiempo 32: Inicia atención para Paciente 2 - Prioridad I: Prioridad absoluta, atención inmediata, tiempo estimado 7 minutos
Tiempo 35: Llega Paciente D - Prioridad I: Prioridad absoluta, atención inmediata
Tiempo 39: Finaliza atenc

Simulación de un centro médico con VARIOS DOCTORES, con posibilidad de que una emergencia interrumpa la atención de un paciente de menor prioridad

In [ ]:
import heapq
import random
import time
import matplotlib.pyplot as plt

# Recolección de datos
tiempos_espera = []
tiempos_atencion_por_prioridad = { "I": [], "II": [], "III": [], "IV": [] }
pacientes_en_espera_tiempo = []
doctores_atenciones = {}

# Definimos las prioridades de triaje
PRIORIDADES = {
    "I": {"descripcion": "Prioridad absoluta, atención inmediata", "demora_maxima": 0},
    "II": {"descripcion": "Muy urgente, riesgo vital", "demora_maxima": 15},
    "III": {"descripcion": "Urgente pero estable, potencial riesgo vital", "demora_maxima": 60},
    "IV": {"descripcion": "Urgencia menor, sin riesgo vital", "demora_maxima": 120}
}

# Clase para representar a un paciente
class Paciente:
    def __init__(self, nombre, prioridad):
        self.nombre = nombre
        self.prioridad = prioridad
        self.tiempo_llegada_simulacion = None  # Tiempo de llegada a la simulación

    def __lt__(self, other):
        return (self.prioridad, self.tiempo_llegada_simulacion) < (other.prioridad, other.tiempo_llegada_simulacion)

    def __str__(self):
        return f"{self.nombre} - Prioridad {self.prioridad}: {PRIORIDADES[self.prioridad]['descripcion']}"

# Clase para manejar eventos en la simulación
class Evento:
    def __init__(self, tiempo, tipo, paciente=None, doctor=None):
        self.tiempo = tiempo  # Momento en el que ocurre el evento
        self.tipo = tipo  # "llegada", "inicio_atencion", "fin_atencion"
        self.paciente = paciente
        self.doctor = doctor

    def __lt__(self, other):
        return self.tiempo < other.tiempo

# Clase para representar un doctor (servidor)
class Doctor:
    def __init__(self, id_doctor):
        self.id_doctor = id_doctor
        self.ocupado = False
        self.paciente_actual = None
        self.tiempo_fin_atencion = 0

    def __str__(self):
        return f"Doctor {self.id_doctor}"

# Clase para el centro de salud multicanal que maneja los eventos y la cola de pacientes
class CentroSaludMulticanal:
    def __init__(self, num_doctores, politica_interrupcion=True):
        self.cola_pacientes = []
        self.eventos = []
        self.tiempo_actual = 0
        self.politica_interrupcion = politica_interrupcion
        self.doctores = [Doctor(i) for i in range(num_doctores)]
        for doctor in self.doctores:
            doctores_atenciones[doctor.id_doctor] = 0  # Inicializar conteo de atenciones por doctor

    def agendar_evento(self, evento):
        heapq.heappush(self.eventos, evento)

    def llegada_paciente(self, paciente):
        print(f"Tiempo {self.tiempo_actual}: Llega {paciente}")
        
        # Registrar tiempo de llegada para cálculo de espera
        paciente.tiempo_llegada_simulacion = self.tiempo_actual
        
        # Buscar un doctor disponible
        doctor_libre = next((doctor for doctor in self.doctores if not doctor.ocupado), None)

        if doctor_libre:
            # Si hay un doctor libre, iniciamos atención inmediata
            self.iniciar_atencion(paciente, doctor_libre)
        else:
            # Si todos los doctores están ocupados, el paciente espera en la cola
            heapq.heappush(self.cola_pacientes, paciente)
        
        # Guardar el número de pacientes en espera en el tiempo actual
        pacientes_en_espera_tiempo.append((self.tiempo_actual, len(self.cola_pacientes)))

    def iniciar_atencion(self, paciente, doctor):
        doctor.ocupado = True
        doctor.paciente_actual = paciente
        tiempo_atencion = random.randint(5, 20)  # Simula tiempo de atención
        doctor.tiempo_fin_atencion = self.tiempo_actual + tiempo_atencion

        # Calcular y guardar el tiempo de espera del paciente
        tiempo_espera = self.tiempo_actual - paciente.tiempo_llegada_simulacion
        tiempos_espera.append(tiempo_espera)
        
        # Guardar el tiempo de atención por prioridad
        tiempos_atencion_por_prioridad[paciente.prioridad].append(tiempo_atencion)
        
        # Contar la atención realizada por el doctor
        doctores_atenciones[doctor.id_doctor] += 1

        print(f"Tiempo {self.tiempo_actual}: {doctor} inicia atención para {paciente}, tiempo estimado {tiempo_atencion} minutos")
        self.agendar_evento(Evento(doctor.tiempo_fin_atencion, "fin_atencion", paciente, doctor))

    def fin_atencion(self, doctor):
        paciente = doctor.paciente_actual
        print(f"Tiempo {self.tiempo_actual}: {doctor} finaliza atención para {paciente}")
        doctor.ocupado = False
        doctor.paciente_actual = None
        # Si hay más pacientes en la cola, atender al siguiente
        if self.cola_pacientes:
            siguiente_paciente = heapq.heappop(self.cola_pacientes)
            self.iniciar_atencion(siguiente_paciente, doctor)

    def procesar_eventos(self, tiempo_maximo):
        while self.eventos and self.tiempo_actual <= tiempo_maximo:
            evento = heapq.heappop(self.eventos)
            self.tiempo_actual = evento.tiempo
            if evento.tipo == "llegada":
                self.llegada_paciente(evento.paciente)
            elif evento.tipo == "fin_atencion":
                self.fin_atencion(evento.doctor)

    def simular(self, tiempo_maximo):
        # Genera eventos de llegada de pacientes de manera aleatoria en el tiempo
        nombres = ["Paciente A", "Paciente B", "Paciente C", "Paciente D", "Paciente E"]
        prioridades = ["I", "II", "III", "IV"]
        
        for i in range(20):  # Genera algunos pacientes de prueba
            tiempo_llegada = random.randint(1, tiempo_maximo)
            nombre = random.choice(nombres)
            prioridad = random.choice(prioridades)
            paciente = Paciente(nombre, prioridad)
            self.agendar_evento(Evento(tiempo_llegada, "llegada", paciente))
        
        self.procesar_eventos(tiempo_maximo)

# Simulación
tiempo_simulacion = 300  # Simular 300 minutos
num_doctores = 3  # Número de doctores disponibles (servidores)
centro_salud = CentroSaludMulticanal(num_doctores, politica_interrupcion=True)
centro_salud.simular(tiempo_simulacion)

Tiempo 11: Llega Paciente A - Prioridad I: Prioridad absoluta, atención inmediata
Tiempo 11: Doctor 0 inicia atención para Paciente A - Prioridad I: Prioridad absoluta, atención inmediata, tiempo estimado 10 minutos
Tiempo 21: Doctor 0 finaliza atención para Paciente A - Prioridad I: Prioridad absoluta, atención inmediata
Tiempo 29: Llega Paciente A - Prioridad I: Prioridad absoluta, atención inmediata
Tiempo 29: Doctor 0 inicia atención para Paciente A - Prioridad I: Prioridad absoluta, atención inmediata, tiempo estimado 5 minutos
Tiempo 34: Doctor 0 finaliza atención para Paciente A - Prioridad I: Prioridad absoluta, atención inmediata
Tiempo 64: Llega Paciente C - Prioridad II: Muy urgente, riesgo vital
Tiempo 64: Doctor 0 inicia atención para Paciente C - Prioridad II: Muy urgente, riesgo vital, tiempo estimado 10 minutos
Tiempo 74: Doctor 0 finaliza atención para Paciente C - Prioridad II: Muy urgente, riesgo vital
Tiempo 82: Llega Paciente A - Prioridad I: Prioridad absoluta, at